# Simple ML introducion

Written by:
- Manuel Szewc (School of Physics, University of Cincinnati)
- Philip Ilten (School of Physics, University of Cincinnati)
$\renewcommand{\gtrsim}{\raisebox{-2mm}{\hspace{1mm}$\stackrel{>}{\sim}$\hspace{1mm}}}\renewcommand{\lessim}{\raisebox{-2mm}{\hspace{1mm}$\stackrel{<}{\sim}$\hspace{1mm}}}\renewcommand{\as}{\alpha_{\mathrm{s}}}\renewcommand{\aem}{\alpha_{\mathrm{em}}}\renewcommand{\kT}{k_{\perp}}\renewcommand{\pT}{p_{\perp}}\renewcommand{\pTs}{p^2_{\perp}}\renewcommand{\pTe}{\p_{\perp\mrm{evol}}}\renewcommand{\pTse}{\p^2_{\perp\mrm{evol}}}\renewcommand{\pTmin}{p_{\perp\mathrm{min}}}\renewcommand{\pTsmim}{p^2_{\perp\mathrm{min}}}\renewcommand{\pTmax}{p_{\perp\mathrm{max}}}\renewcommand{\pTsmax}{p^2_{\perp\mathrm{max}}}\renewcommand{\pTL}{p_{\perp\mathrm{L}}}\renewcommand{\pTD}{p_{\perp\mathrm{D}}}\renewcommand{\pTA}{p_{\perp\mathrm{A}}}\renewcommand{\pTsL}{p^2_{\perp\mathrm{L}}}\renewcommand{\pTsD}{p^2_{\perp\mathrm{D}}}\renewcommand{\pTsA}{p^2_{\perp\mathrm{A}}}\renewcommand{\pTo}{p_{\perp 0}}\renewcommand{\shat}{\hat{s}}\renewcommand{\a}{{\mathrm a}}\renewcommand{\b}{{\mathrm b}}\renewcommand{\c}{{\mathrm c}}\renewcommand{\d}{{\mathrm d}}\renewcommand{\e}{{\mathrm e}}\renewcommand{\f}{{\mathrm f}}\renewcommand{\g}{{\mathrm g}}\renewcommand{\hrm}{{\mathrm h}}\renewcommand{\lrm}{{\mathrm l}}\renewcommand{\n}{{\mathrm n}}\renewcommand{\p}{{\mathrm p}}\renewcommand{\q}{{\mathrm q}}\renewcommand{\s}{{\mathrm s}}\renewcommand{\t}{{\mathrm t}}\renewcommand{\u}{{\mathrm u}}\renewcommand{\A}{{\mathrm A}}\renewcommand{\B}{{\mathrm B}}\renewcommand{\D}{{\mathrm D}}\renewcommand{\F}{{\mathrm F}}\renewcommand{\H}{{\mathrm H}}\renewcommand{\J}{{\mathrm J}}\renewcommand{\K}{{\mathrm K}}\renewcommand{\L}{{\mathrm L}}\renewcommand{\Q}{{\mathrm Q}}\renewcommand{\R}{{\mathrm R}}\renewcommand{\T}{{\mathrm T}}\renewcommand{\W}{{\mathrm W}}\renewcommand{\Z}{{\mathrm Z}}\renewcommand{\bbar}{\overline{\mathrm b}}\renewcommand{\cbar}{\overline{\mathrm c}}\renewcommand{\dbar}{\overline{\mathrm d}}\renewcommand{\fbar}{\overline{\mathrm f}}\renewcommand{\pbar}{\overline{\mathrm p}}\renewcommand{\qbar}{\overline{\mathrm q}}\renewcommand{\rbar}{\overline{\mathrm{r}}}\renewcommand{\sbar}{\overline{\mathrm s}}\renewcommand{\tbar}{\overline{\mathrm t}}\renewcommand{\ubar}{\overline{\mathrm u}}\renewcommand{\Bbar}{\overline{\mathrm B}}\renewcommand{\Fbar}{\overline{\mathrm F}}\renewcommand{\Qbar}{\overline{\mathrm Q}}\renewcommand{\tms}{{t_{\mathrm{\tiny MS}}}}\renewcommand{\Oas}[1]{{\mathcal{O}\left(\as^{#1}\right)}}$

## Introduction

This notebook wants to implement simple Machine Learning algorithms for classification

In [ ]:
import os

# To generate data and handle arrays
import numpy as np

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

## Classification

In classification problems, we are interested in predicting a class asignment $\mathcal{C}_{k}$ to a measurement $x$. The training data consists of paired measurements and class labels $x_{\text{train}},t_{\text{train}}$.

There are three types of classification approaches:

* Discriminant functions, where we learn a function $y(x,w)$ that defines the border between classes. For the binary case, and using $t \in \{-1,1\}$ , this corresponds to definen a threshold $y_{0}$ (usually zero) such that if $y(x,w)>y_{0}$, $t = 1$ and $t=-1$ otherwise. This is the case for `Perceptron` and `Support Vector Machines` among others. `DecisionTrees` can be thought of in these terms as well, although they can also estimate probabilities.
* Discriminative models, where we learn a function $y(x,w)$ that encodes the probability of a class asignment given $x$, $p(\mathcal{C}_{k}|x)$. A class asignment can be made by selecting the class which maximizes the probability, although other criteria can be applied as well. `LogisticRegressors` and simple `Neural Network Classifiers` are examples of this.
* Generative models, where we learn a function $y(x,w)$ that encodes the probability of $x$ per class, $p(x|\mathcal{C}_{k})$. A class asignment can be made by estimating the per class probabilities using Bayes' rule and selecting the class which maximizes this probability. `Naive Bayes` classifiers or conditional density estimators are examples of this.


Let's focus more on discriminative models. Here, we are interested in the per-class probability, which is a **posterior** over class asignments.  For the binary case, we only need to specifiy $p(\mathcal{C}_{1}|x)$ since $p(\mathcal{C}_{2}|x)=1-p(\mathcal{C}_{1}|x)$. We deal with the multiclass problem further down the notebook.

For the binary case, we compute

$$p(\mathcal{C}_{1}|x)=\frac{p(x|\mathcal{C}_{1})p(\mathcal{C}_{1})}{p(x|\mathcal{C}_{1})p(\mathcal{C}_{1})+p(x|\mathcal{C}_{2})p(\mathcal{C}_{2})}=\frac{1}{1+\frac{p(x|\mathcal{C}_{2})p(\mathcal{C}_{2})}{p(x|\mathcal{C}_{1})p(\mathcal{C}_{1})}}$$

Defining the **log-odds ratio**

$$a=\text{Ln }\frac{p(x|\mathcal{C}_{1})p(\mathcal{C}_{1})}{p(x|\mathcal{C}_{2})p(\mathcal{C}_{2})}=\text{Ln }\frac{p(\mathcal{C}_{1}|x)}{p(\mathcal{C}_{2}|x)}$$

we have that

$$p(\mathcal{C}_{1}|x)=\frac{1}{1+\text{e}^{-a}}=\sigma(a)$$

where $\sigma(a)$ is the  **sigmoid function**. Thus, the binary problem is reduced to computing the log-odds ratio between the two classes.

In [ ]:
prob = np.linspace(-10, 10, 200)

a_vals = 1 / (1 + np.exp(-prob))

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(prob, a_vals, color="black")
ax.scatter(0.0, 0.5, color="red")
ax.axvline(0.0, linestyle="dashed", color="red")
ax.axhline(0.5, linestyle="dashed", color="red")
ax.set_ylabel(r"$p(\mathcal{C}_{1}|x)$", fontsize=16)
ax.set_xlabel("Log odds", fontsize=16)

We see how the sigmoid function is a function lying in $[0,1]$, even as the log odds ratio takes values in $(-\infty,\infty)$. This is necessary since we're interpreting it as a probability.

A particularly important point is where the two classes are equally likely. There, the log odds ratio is 1 and the per class probability is 0.5, as it should. In general, this is selected as the decision boundary.

Two useful properties of the sigmoid function are:

$$\sigma(-a)=1-\sigma(a)$$

and

$$\frac{d\sigma}{da}=\sigma(1-\sigma)$$

For the $K$ class case, we can generalize the sigmoid to the normalized exponential or **softmax** function:

$$p(\mathcal{C}_{k}|x)=\frac{p(x|\mathcal{C}_{k})p(\mathcal{C}_{k})}{\sum_{l=1}^{K}p(x|\mathcal{C}_{l})p(\mathcal{C}_{l})}=\frac{e^{a_{k}}}{\sum_{l=1}^{K}e^{a_{l}}}$$

where $a_{k}=\text{Ln }p(x|\mathcal{C}_{k})p(\mathcal{C}_{k})$

### Logistic Regression

The simplest discriminative classifier is the `Logistic Regressor`. For the two class case, we model the per class probability posterior as

$$p(\mathcal{C}_{1}|\vec{x})=y(\vec{x},\vec{w})=\sigma(\vec{w}^{T}\vec{\phi}(\vec{x}))$$

$$p(\mathcal{C}_{0}|\vec{x})=1-y(\vec{x},\vec{w})=1-\sigma(\vec{w}^{T}\vec{\phi}(\vec{x}))=\sigma(-\vec{w}^{T}\vec{\phi}(\vec{x}))$$

We already have two features of note:

- $\vec{w}^{T}\vec{\phi}(\vec{x})$ acts as the decision function in a discriminant approach. In particular, $\vec{w}^{T}\vec{\phi}(\vec{x})=0$ defines the equi-probability surface.
- The sigmoid is a **non-linear activation function**. This feature, necessary to transform a linear model into a probability estimate, will be very present in `Neural Networks` even for regression for different reasons (the universal approximation is ensured by internal non-linear activation functions between neuron layers).

As in the generalized linear models for regression,  $\vec{\phi}$ includes in principle the bias with $\phi_{0}(\vec{x})=1$.

As with regression, we can group all training pairs into a design matrix

$$\Phi=\begin{pmatrix}\vec{\phi}^{T}(\vec{x}_{1}) \\ ... \\ \vec{\phi}^{T}(\vec{x}_{N})\end{pmatrix}$$

obtaining

$$\sigma(\Phi\cdot \vec{w}) = \begin{pmatrix}\sigma(\vec{\phi}^{T}(\vec{x}_{1}) \cdot\vec{w})\\ ... \\ \sigma(\vec{\phi}^{T}(\vec{x}_{N})\cdot\vec{w})\end{pmatrix}=\begin{pmatrix}\sigma((\vec{w}^{T} \cdot\vec{\phi}(\vec{x}_{1}))^T)\\ ... \\ \sigma((\vec{w}^{T} \cdot\vec{\phi}(\vec{x}_{N}))^T)\end{pmatrix}=\begin{pmatrix}y(\vec{x}_{1},\vec{w})\\ ... \\ y(\vec{x}_{N},\vec{w})\end{pmatrix}$$

Now we need a criteria to obtain the best coefficients $\vec{w}$. As in regression, a natural choice is to maximize the likelhiood (or minmize the negative log-likelihood). For a binary variable, we have only two possible results which we can label as success or failure or heads and tails. Assuming the points are independent and identically distributed, each label has a Bernoulli distribution

$$p(\text{t}|\vec{x},\vec{w})=p(t|\mu)=\mu^{t}(1-\mu)^{1-t}$$

where $\mu=p(\mathcal{C}_{1}|\vec{x})$ is the success probability, which is exactly what we're trying to model!

Thus, for a dataset $\vec{x}_{n}$, with $n=1,..,N$ the likelihood is

$$\mathcal{L}(\vec{w})=p(\text{T}|X,\vec{w})=\prod_{n=1}^{N}y^{t_n}_{n}(1-y_{n})^{1-t_n}$$

where $y_{n}=y(x_{n},\vec{w})$. Again, the logarithm is easier to handle

$$\ln \mathcal{L}(\vec{w})=\sum_{n=1}^{N}(t_{n}\ln y_{n}+(1-t_{n})\ln (1-y_{n}))$$

Again we can define an error by taking the negative log-likelihood. This is called the **binary cross-entropy** (BCE) between $t$ and $y$

$$E(\vec{w})=-\sum_{n=1}^{N}(t_{n}\ln y_{n}+(1-t_{n})\ln (1-y_{n}))$$

We can see how BCE forces the right behaviour into the model:

If $t_{n}=1$, we care about $\ln y_{n}$. Thus, $y_{n}$ needs to be close to 1 to minimize the BCE.

If $t_{n}=0$, we care about $\ln (1-y_{n})$. Thus, $y_{n}$ needs to be close to 0 to minimize the BCE.

Just a reminder, $y_{n}$ is not meant to match $t_{n}$ but $p(t_{n}|\vec{x}_{n},\vec{w})$. Thus, we also seek a degree of confidence to be embedded into the model.


To obtain useful estimates, we need to be able to minimize the BCE with respect to the parameters $\vec{w}$.

For logistic regression, the problem is not as easy as it was for linear regression due to the presence of the logarithms and the sigmoid function. However, we can still solve this numerically. A particularly good choice is the Iterative Reweighted Least Squares or IRLS, which considers a Newton-Ralphson update for step $i$:

$$\vec{w}^{i}=\vec{w}^{i-1}-\mathrm{H}^{-1}\nabla E(\vec{w})|_{\vec{w}^{i-1}}$$

where $\vec{w}^{0}$ is the initialized vector of parameters, $\nabla E(\vec{w})$ is the BCE gradient with respect to the parameters and $\mathrm{H}$ is the Hessian matrix. For a logistic regressor where we combine a linear model with a sigmoide function, this simplifies to a set of **iterative normal equations**:

$$\vec{w}^{i}=\Phi^{T}\mathrm{R}\Phi^{-1}\Phi^{T}\mathrm{R}\mathrm{z}$$

where $\Phi$ is the design matrix, $\mathrm{R}$ a diagonal matrix whose elements are $y_{n}(1-y_{n})$ and $\mathrm{z}$ is

$\mathrm{z}=\Phi\vec{w}^{i-1}-\mathrm{R}^{-1}(\mathrm{Y}-\mathrm{T})$

where $\mathrm{Y}$ e $\mathrm{T}$ are the prediction and label vectors. One should remember that $\mathrm{R}$, $\mathrm{z}$ and $\mathrm{Y}$ are all functions of $\vec{w}^{-1}$.


We could have applied such an algorithm to linear regression, and observed how it converge to the closed solution in 1 step. Additionally, be sure to notice that is an iterative algorithm but it is not sequential since it considers the full dataset at each step.

### Exercise

Let's consider the following dataset

In [ ]:
size1 = 250
mu1 = [0, 0]
cov1 = [[1, 0.95], [0.95, 1]]

size2 = 200
mu2 = [-1, 0.5]
cov2 = [[1, 0.8], [0.8, 1]]

np.random.seed(20200922)
# Sample classes
xc1 = np.random.multivariate_normal(mean=mu1, cov=cov1, size=size1).T
xc2 = np.random.multivariate_normal(mean=mu2, cov=cov2, size=size2).T

print(xc1.shape, xc2.shape)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)

ax.plot(*xc1, "ob", mfc="None", label="C1")
ax.plot(*xc2, "or", mfc="None", label="C2")

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")
ax.legend(loc="lower right", fontsize=16)
ax.set_aspect("equal")

In [ ]:
X = np.hstack([xc1, xc2]).T

tc1 = np.ones(xc1.shape[1])
tc2 = np.zeros(xc2.shape[1])

t = np.concatenate([tc1, tc2]).reshape(-1, 1)

Using this sigmoid function

In [ ]:
def sigmoid(logoddsvec):
    return 1 / (1 + np.exp(-logoddsvec))

Train a Logistic Regressor, print the best parameters, compute the predicted probability per measurement and plot it as a function of ($x_1,x_2$).

Veamos como evolucionan los coeficientes

In [ ]:
W = w[-1]
y = sigmoid(np.dot(Phi, W))
fig = plt.figure(figsize=(10, 10))
plt.hist(
    y[t[:, 0] > 0.5],
    bins=np.linspace(0, 1, 10),
    density=True,
    alpha=0.5,
    color="blue",
    label="Predicted probabilities for $C_1$",
)
plt.hist(
    y[t[:, 0] <= 0.5],
    bins=np.linspace(0, 1, 10),
    density=True,
    alpha=0.5,
    color="red",
    label="Predicted probabilities for $C_2$",
)
plt.axvline(
    0.5, color="red", linestyle="dashed", linewidth=2, label="Decision boundary"
)
plt.xlabel("Predicted probability")
plt.ylabel("Density")
plt.title("Distribution of Predicted Probabilities")
plt.legend()
plt.show()

In [ ]:
###STOP_SOLUTION

In [ ]:
def accuracy_score(labels, predictions):
    return np.mean([a == b for a, b in zip(labels, predictions)])


def precision_score(labels, predictions):
    return np.sum([a and b for a, b in zip(labels == 1, predictions == 1)]) / np.sum(
        predictions == 1
    )


def recall_score(labels, predictions):
    return np.sum([a and b for a, b in zip(labels == 1, predictions == 1)]) / np.sum(
        labels == 1
    )

Let's check them out

In [ ]:
print(accuracy_score(t, y_pred))  # Accuracy
print(precision_score(t, y_pred))  # Precision
print(recall_score(t, y_pred))  # Recall

In [ ]:
cf = confusion_matrix(t, y_pred)
print(cf)

Introduce *varying metric *The ROC curve

### Multiclass (`sklearn`-based)

Ejemplifiquemos con un caso de juguete:

In [ ]:
plt.scatter(X[t == 0, 0], X[t == 0, 1], c="red", label="$C_{0}$")
plt.scatter(X[t == 1, 0], X[t == 1, 1], c="blue", label="$C_{1}$")
plt.legend(loc="upper left")

In [ ]:
from sklearn.linear_model import LogisticRegression, Perceptron

lr = LogisticRegression()
percep = Perceptron()

Para el caso del Perceptron, tendremos una única función de decisión. El criterio usual es que si $y\geq0$, asigno a la clase $C_{1}$. Esto se puede modificar tal como vimos antes.

In [ ]:
x1vals = np.linspace(np.min(X[:, 0]), np.max(X[:, 0]), 100)
x2vals = np.linspace(np.min(X[:, 1]), np.max(X[:, 1]), 100)
X1, X2 = np.meshgrid(x1vals, x2vals)
Xvals = np.array([X1.ravel(), X2.ravel()]).T
Z = percep.decision_function(Xvals).reshape(X1.shape)
plt.contourf(X1, X2, Z, levels=[-1000, 0.0, 1000])
plt.colorbar()
plt.scatter(X[t == 0, 0], X[t == 0, 1], c="red", label="$C_{0}$")
plt.scatter(X[t == 1, 0], X[t == 1, 1], c="blue", label="$C_{1}$")
plt.legend(loc="upper left")

In [ ]:
print(lr.predict_proba(X[:1]))

Vayamos ahora al caso multiclase:
- Tenemos las etiquetas, que pueden venir esencialmente en dos formas: {0,1,...,K-1},{[1,0,...,0],[0,1,0,...,0],...}. Para `sklearn`, con usar la primera estamos perfecto. Si codeamos a mano, hay que tener cuidado.
- Tenemos las predicciones. Acá es donde aparece la gran diferencia. ¿Qué entrenamos?
- El primer enfoque es el de _uno contra el resto_. Para cada clase $K$, entreno un clasificador que aprenda a distinguir esa clase del resto. Tenemos entonces K clasificadores. Por ejemplo, para 4 clases: (0,123), (1,023), (2,013), (3,012). Uno asigna entonces una clase viendo como se intersectan las distintas regiones de los clasificadores. Eso introduce la posibilidad de regiones ambiguas. Además, el problema se puede volver muy desbalanceado.
- El segundo enfoque es el de _uno contra uno_. En este enfoque, entreno un clasificador para cada combinación de dos clases. Tenemos entonces $\frac{K(K-1)}{2}$ clasificadores, uno por cada problema de dos clases. Por ejemplo, para 4 clases: (0,1), (0,2), (0,3), (1,2), (1,3), (2,3). Para asignar una clase, tomamos entonces el voto mayoritario. Es decir, cual es la clase que recibe más votos de cada uno de estos clasificadores. Esto también introduce ambigüedades.
- El tercer enfoque, que no asigna ambiguedades. Es aprender un único clasificador de $K$ clases. Para el caso de función discriminante, esto se logra aprendiendo K funciones $y_{k}$ al mismo tiempo, y asignando la clase como la que da el maximo $y_{k}$ entre todos. Las fronteras de decisión son entonces dadas por las regiones donde coinciden dos funciones. Se puede mostrar que con este criterio ya no hay ambigüedades. Para el caso de modelos discriminativos y generativos, esto es todavia más fácil. ¡Aprendemos $p(C_{k}|x)$ tal como antes! Ahora simplemente no simplificamos la clase redundante. Tampoco hay ambigüedades.

Cual de los tres enfoques usamos depende del algoritmo. En particular, hay algunos que no pueden utilizar el tercer enfoque y entonces aparece el problema. Noten que ahora escribi el criterio, mientras que antes siempre aclare que podiamos jugar con el umbral. Para el caso multiclase, el análisis que hicimos se vuelve más díficil. Para poder hacerlo, vamos a binarizar el problema.

In [ ]:
from sklearn.datasets import make_blobs

gt_center = np.array([[3.0, 3.0], [-3.0, -3.0], [-3.0, 3.0], [3.0, -3.0]])
X, t = make_blobs(
    1000,
    n_features=2,
    centers=gt_center,
    cluster_std=1.0,
    random_state=1234,
)

Entrenemos una función discriminante y un modelo discriminativo:

In [ ]:
lr.fit(X, t)
lda.fit(X, t)

In [ ]:
print(lda.decision_function(X[:1]))

In [ ]:
x1vals = np.linspace(np.min(X[:, 0]), np.max(X[:, 0]), 100)
x2vals = np.linspace(np.min(X[:, 1]), np.max(X[:, 1]), 100)
X1, X2 = np.meshgrid(x1vals, x2vals)
Xvals = np.array([X1.ravel(), X2.ravel()]).T
Z = np.argmax(lda.decision_function(Xvals), axis=1).reshape(X1.shape)
plt.contourf(X1, X2, Z, cmap="plasma", levels=[-0.5, 0.5, 1.5, 2.5, 3.5])
plt.colorbar()
plt.scatter(X[t == 0, 0], X[t == 0, 1], c="red", label="$C_{0}$")
plt.scatter(X[t == 1, 0], X[t == 1, 1], c="blue", label="$C_{1}$")
plt.scatter(X[t == 2, 0], X[t == 2, 1], c="limegreen", label="$C_{2}$")
plt.scatter(X[t == 3, 0], X[t == 3, 1], c="salmon", label="$C_{3}$")
plt.legend(loc="upper left")

In [ ]:
print(lr.predict_proba(X[:1]))

In [ ]:
x1vals = np.linspace(np.min(X[:, 0]), np.max(X[:, 0]), 100)
x2vals = np.linspace(np.min(X[:, 1]), np.max(X[:, 1]), 100)
X1, X2 = np.meshgrid(x1vals, x2vals)
Xvals = np.array([X1.ravel(), X2.ravel()]).T
Z = np.argmax(lr.predict_proba(Xvals), axis=1).reshape(X1.shape)
plt.contourf(X1, X2, Z, cmap="plasma", levels=[-0.5, 0.5, 1.5, 2.5, 3.5])
plt.colorbar()
plt.scatter(X[t == 0, 0], X[t == 0, 1], c="red", label="$C_{0}$")
plt.scatter(X[t == 1, 0], X[t == 1, 1], c="blue", label="$C_{1}$")
plt.scatter(X[t == 2, 0], X[t == 2, 1], c="limegreen", label="$C_{2}$")
plt.scatter(X[t == 3, 0], X[t == 3, 1], c="salmon", label="$C_{3}$")
plt.legend(loc="upper left")

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(t, lr.predict(X)))
print(accuracy_score(t, lr.predict(X)))

In [ ]:
from sklearn.metrics import precision_score, f1_score

for k in range(4):
    print("Clase " + str(k) + " contra todos")
    print(f1_score(np.where(t == k, 1.0, 0.0), np.where(lr.predict(X) == k, 1.0, 0.0)))

In [ ]:
from sklearn.svm import LinearSVC, SVC

lsvc_1 = LinearSVC(loss="hinge", penalty="l2", C=10.0, max_iter=1000)
svc_1 = SVC(kernel="linear", C=10.0, decision_function_shape="ovr")
svc_2 = SVC(kernel="linear", C=10.0, decision_function_shape="ovo")

In [ ]:
print(lsvc_1.decision_function(X[:1]).shape)
print(svc_1.decision_function(X[:1]).shape)
print(svc_2.decision_function(X[:1]).shape)

In [ ]:
print(lsvc_1.coef_.shape)  # (nclasses, ncoefs)
print(svc_1.coef_.shape)  # (nclasses*(nclasses-1)/2, ncoefs)
print(svc_2.coef_.shape)  # (nclasses*(nclasses-1)/2, ncoefs)

In [ ]:
print(np.allclose(svc_1.coef_, svc_2.coef_))

In [ ]:
x1vals = np.linspace(np.min(X[:, 0]), np.max(X[:, 0]), 100)
x2vals = np.linspace(np.min(X[:, 1]), np.max(X[:, 1]), 100)
X1, X2 = np.meshgrid(x1vals, x2vals)
Xvals = np.array([X1.ravel(), X2.ravel()]).T
Z = lsvc_1.predict(Xvals).reshape(X1.shape)
plt.contourf(X1, X2, Z, cmap="plasma", levels=[-0.5, 0.5, 1.5, 2.5, 3.5])
plt.colorbar()
plt.scatter(X[t == 0, 0], X[t == 0, 1], c="red", label="$C_{0}$")
plt.scatter(X[t == 1, 0], X[t == 1, 1], c="blue", label="$C_{1}$")
plt.scatter(X[t == 2, 0], X[t == 2, 1], c="limegreen", label="$C_{2}$")
plt.scatter(X[t == 3, 0], X[t == 3, 1], c="salmon", label="$C_{3}$")
plt.legend(loc="upper left")

In [ ]:
x1vals = np.linspace(np.min(X[:, 0]), np.max(X[:, 0]), 100)
x2vals = np.linspace(np.min(X[:, 1]), np.max(X[:, 1]), 100)
X1, X2 = np.meshgrid(x1vals, x2vals)
Xvals = np.array([X1.ravel(), X2.ravel()]).T
Z = svc_2.predict(Xvals).reshape(X1.shape)
plt.contourf(X1, X2, Z, cmap="plasma", levels=[-0.5, 0.5, 1.5, 2.5, 3.5])
plt.colorbar()
plt.scatter(X[t == 0, 0], X[t == 0, 1], c="red", label="$C_{0}$")
plt.scatter(X[t == 1, 0], X[t == 1, 1], c="blue", label="$C_{1}$")
plt.scatter(X[t == 2, 0], X[t == 2, 1], c="limegreen", label="$C_{2}$")
plt.scatter(X[t == 3, 0], X[t == 3, 1], c="salmon", label="$C_{3}$")
plt.scatter(
    svc_2.support_vectors_[:, 0],
    svc_2.support_vectors_[:, 1],
    marker="+",
    color="black",
    label="SV",
)

plt.legend(loc="upper left")